In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
import os
import shutil
import zipfile
import random
import regex as re
import numpy as np

In [ ]:
def extract_data(zip_folders):
  """
  Extracts images and labels from zip files
  """
  images = []
  image_labels = []
  for zip_folder in zip_folders:
    with zipfile.ZipFile(zip_folder + '.zip', 'r') as zip_ref:
      filenames = zip_ref.namelist()

      for filename in filenames:
        images.append(filename)
        # Get BOD5 value from filename
        filename = os.path.splitext(filename)
        tokens = re.split('_', filename[0])
        value = float(tokens[-1])
        image_labels.append(value)

  return images, image_labels

In [ ]:
train_zips = ["/content/drive/MyDrive/Training_Set_1", "/content/drive/MyDrive/Training_Set_2"
,"/content/drive/MyDrive/Training_Set_3", "/content/drive/MyDrive/Training_Set_4"
,"/content/drive/MyDrive/Training_Set_5", "/content/drive/MyDrive/Training_Set_6"
,"/content/drive/MyDrive/Training_Set_7"]

val_zip = ["/content/drive/MyDrive/Validation_Set"]

train_images, train_labels = extract_data(train_zips)
val_images, val_labels = extract_data(val_zip)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_images.shape)
print(train_labels.shape)

val_images = np.array(val_images)
val_labels = np.array(val_labels)
print(val_images.shape)
print(val_labels.shape)

In [ ]:
import cv2
import rasterio
import random
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
!apt update && apt install cuda-11-8
pip install deeplabcut[tf]

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found - On for CPU time!')
else:
    print('Found GPU at {}'.format(device_name))

In [ ]:
def extract_file(zip_folder, image_names):
  """
  Extracts and open images into local Colab Directory
  """
  batch_images = []
  with zipfile.ZipFile(zip_folder + '.zip', 'r') as zip_ref:
    for image_name in image_names:
      zip_ref.extract(image_name, '/content/temp_dataset')
      image_path = os.path.join('/content/temp_dataset', image_name)
      with rasterio.open(image_path, 'r') as src:
        image = src.read()
        image = np.transpose(image, (1,2,0))
        image = cv2.resize(image, (512,512))
        image[np.isnan(image)] = 0
        batch_images.append(image)

    shutil.rmtree('/content/temp_dataset')
    return batch_images

class DataGenerator(keras.utils.Sequence):
  def __init__(self, image_names, labels, zip_folders, batch_size, **kwargs):
    super().__init__(**kwargs)
    self.image_names = image_names
    self.labels = labels
    self.zip_folders = zip_folders
    self.batch_size = batch_size
    self.folder_idx = 0;

  def __len__(self):
    return len(self.image_names) // self.batch_size

  def __getitem__(self, idx):

    image_sublist = self.image_names[idx * self.batch_size : (idx + 1) * self.batch_size]
    if len(self.zip_folders) > 1:
        folder_idx = (idx*self.batch_size) // 1024
    else:
        folder_idx = 0

    batch_images = extract_file(self.zip_folders[folder_idx], image_sublist)
    batch_labels = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

    np_images = np.array(batch_images)
    np_labels = np.array(batch_labels)

    return np_images, np_labels

In [ ]:
batch_size = 64

train_dataset = DataGenerator(train_images, train_labels, train_zips, batch_size)
val_dataset = DataGenerator(val_images, val_labels, val_zip, batch_size)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(644,644,12)),
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.Conv2D(24, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Conv2D(48, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Conv2D(96, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(108, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1, activation=None)
])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=False)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.R2Score()])

In [ ]:
os.makedirs('/content/drive/MyDrive/model_checkpoints', exist_ok=True)
save_path = os.path.join('/content/drive/MyDrive/model_checkpoints', 'model_aug.keras')

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path,
                                                      monitor='val_loss',
                                                      verbose=0,
                                                      save_best_only=False,
                                                      save_weights_only=False,
                                                      save_freq='epoch')

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=100,
                    batch_size=batch_size,
                    verbose=1,
                    callbacks=[early_stopping, lr_reduce, model_checkpoint])

In [ ]:
def training_plot(metrics, history):
    f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
    for idx, metric in enumerate(metrics):
        ax[idx].plot(history.history[metric], ls='dashed')
        ax[idx].set_xlabel('Epochs')
        ax[idx].set_ylabel(metric)
        ax[idx].plot(history.history['val_'+metric]);
        ax[idx].legend(['train_'+metric, 'val_'+metric])

In [ ]:
training_plot(['loss', 'r2_score'], history)

In [ ]:
os.makedirs('/content/drive/MyDrive/model_weights', exist_ok=True)
path = os.path.join('/content/drive/MyDrive/model_weights', 'model_weights.h5')
model.save_weights(path)